In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import hinge_loss, mean_absolute_error,mean_squared_log_error,median_absolute_error,mean_absolute_percentage_error,mean_poisson_deviance

In [3]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.4, random_state=44)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1411


In [4]:

def get_metrics(y, p_y):
    metrics = {}
    try:
        metrics['mse'] = mse(y,p_y)
    except:
        pass
    try:
        metrics['hinge_loss'] = hinge_losss(y,p_y)
    except:
        pass
    try:
        metrics['mean_absolute_error'] = mean_absolute_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_squared_log_error'] = mean_squared_log_error(y,p_y)
    except:
        pass
    try:
        metrics['median_absolute_error'] = median_absolute_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_absolute_percentage_error'] = mean_absolute_percentage_error(y,p_y)
    except:
        pass
    try:
        metrics['mean_poisson_deviance'] = mean_poisson_deviance(y,p_y)
    except:
        pass

    return metrics
    

# Handmade NN

In [5]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [6]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [7]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mae')
model_ann.compile(optimizer='adam', loss='mae')
# model_conv.compile(optimizer='adam', loss='mse')

In [8]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
27/27 [==============================] - 2s 14ms/step - loss: 448.9602
Epoch 2/30
27/27 [==============================] - 0s 14ms/step - loss: 2.5773
Epoch 3/30
27/27 [==============================] - 0s 15ms/step - loss: 0.3252
Epoch 4/30
27/27 [==============================] - 0s 14ms/step - loss: 1.3625
Epoch 5/30
27/27 [==============================] - 0s 14ms/step - loss: 3.8568
Epoch 6/30
27/27 [==============================] - 0s 14ms/step - loss: 1.4235
Epoch 7/30
27/27 [==============================] - 0s 14ms/step - loss: 0.1849
Epoch 8/30
27/27 [==============================] - 0s 14ms/step - loss: 0.1837
Epoch 9/30
27/27 [==============================] - 0s 15ms/step - loss: 0.1836
Epoch 10/30
27/27 [==============================] - 0s 14ms/step - loss: 0.1836
Epoch 11/30
27/27 [==============================] - 0s 14ms/step - loss: 0.1836
Epoch 12/30
27/27 [==============================] - 0s 14ms/step - loss: 0.1836
Epoch 13/30
27/27 [================

In [9]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
27/27 [==============================] - 3s 14ms/step - loss: 0.3446
Epoch 2/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2620
Epoch 3/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2427
Epoch 4/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2105
Epoch 5/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2029
Epoch 6/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2078
Epoch 7/30
27/27 [==============================] - 0s 14ms/step - loss: 0.1985
Epoch 8/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2219
Epoch 9/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2212
Epoch 10/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2231
Epoch 11/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2086
Epoch 12/30
27/27 [==============================] - 0s 14ms/step - loss: 0.2086
Epoch 13/30
27/27 [==================

In [10]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
print(get_metrics(Y_predicted, Y_test_flat))
print(get_metrics(Y_ann_predicted, Y_test_flat))

18/18 [==============================] - 0s 3ms/step
{'mse': 74.69386200220019, 'mean_absolute_error': 0.07032110592560496, 'median_absolute_error': 0.011269081010038052, 'mean_absolute_percentage_error': 4.349572050934206}
{'mse': 74.69409605372209, 'mean_absolute_error': 0.06431526751237207, 'mean_squared_log_error': 0.009424563879682098, 'median_absolute_error': 0.003040958792254606, 'mean_absolute_percentage_error': 5.271553509739742}


In [11]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
# Y_predicted

In [12]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
# Y_ann_predicted

# AutoKeras

In [13]:
import autokeras as ak
from tensorflow.keras.metrics import MeanAbsoluteError
import numpy as np

In [14]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [15]:

input_node = ak.StructuredDataInput()
output_node = ak.DenseBlock()(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(inputs=input_node, outputs=output_node,overwrite=True, max_trials=100,metrics=['mae'],loss='mean_absolute_error',
)
reg.fit(train_set,epochs=11,validation_split=0.60)

# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|

Trial 100 Complete [00h 00m 02s]
val_loss: 11266.4248046875

Best val_loss So Far: 0.07737971842288971
Total elapsed time: 00h 03m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/11
27/27 [==============================] - 1s 2ms/step - loss: 4505.7676 - mae: 4505.7676
Epoch 2/11
27/27 [==============================] - 0s 2ms/step - loss: 1.9914 - mae: 1.9914
Epoch 3/11
27/27 [==============================] - 0s 2ms/step - loss: 3.8131 - mae: 3.8131
Epoch 4/11
27/27 [==============================] - 0s 2ms/step - loss: 0.6493 - mae: 0.6493
Epoch 5/11
27/27 [==============================] - 0s 2ms/step - loss: 0.1836 - mae: 0.1836
Epoch 6/11
27/27 [==============================] - 0s 2ms/step - loss: 0.2600 - mae: 0.2600
Epoch 7/11
27/27 [==============================] - 0s 2ms/step - loss: 3.4854 - mae: 3.4854
Epoch 8/11
27/27 [==============================] - 0s 2ms/step - loss: 0.1945 - mae: 0.1945
Epoch 9/11
27/27 [==============================] - 0s 2ms/step - loss: 0.625

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [16]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y

18/18 [==============================] - 0s 1ms/step - loss: 0.0643 - mae: 0.0643
[0.06433181464672089, 0.06433181464672089]
{'mse': 74.69402003850641, 'mean_absolute_error': 0.06433182287922903, 'mean_squared_log_error': 0.009416884413971045, 'median_absolute_error': 0.0031583952514693337, 'mean_absolute_percentage_error': 1086135949.754243}


array([[0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       ...,
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536]], dtype=float32)

In [17]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True, metrics=['mse'],)
regressor.fit(train_set, epochs=11)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))

Trial 66 Complete [00h 00m 09s]
val_loss: 214.7057647705078

Best val_loss So Far: 212.2715606689453
Total elapsed time: 00h 09m 32s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/11
27/27 [==============================] - 2s 3ms/step - loss: 1519.8741 - mse: 1519.8741
Epoch 2/11
27/27 [==============================] - 0s 3ms/step - loss: 1519.2330 - mse: 1519.2330
Epoch 3/11
27/27 [==============================] - 0s 3ms/step - loss: 1518.0933 - mse: 1518.0933
Epoch 4/11
27/27 [==============================] - 0s 3ms/step - loss: 1516.0715 - mse: 1516.0715
Epoch 5/11
27/27 [==============================] - 0s 3ms/step - loss: 1512.8712 - mse: 1512.8712
Epoch 6/11
27/27 [==============================] - 0s 3ms/step - loss: 1508.2810 - mse: 1508.2810
Epoch 7/11
27/27 [==============================] - 0s 3ms/step - loss: 1502.3467 - mse: 1502.3467
Epoch 8/11
27/27 [==============================] - 0s 3ms/step - loss: 1494.3534 - mse: 1494.3534
Epoch 9/11
27/27 [==============================] - 0s 3ms/step - loss: 1484.8025 - mse: 1484.8025
Epoch 10/11
27/27 [==============================] - 0s 3ms/step - loss: 1473.3467 - mse: 1473.3467
Epoch 11/

INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


18/18 [==============================] - 1s 2ms/step - loss: 82.5904 - mse: 82.5904
[82.59040832519531, 82.59040832519531]
{'mse': 82.5904002112041, 'mean_absolute_error': 0.34257961825636507, 'median_absolute_error': 0.2517173029577706, 'mean_absolute_percentage_error': 58315875040.55665}


In [18]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                4112      
                                                                 
 re_lu_1 (ReLU)              (None, 16)                0         
                                                                 
 dropout_1 (Dropout)         (None, 16)                0     

In [19]:
model_regressor = regressor.export_model()
model_regressor.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 128)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 128)              257       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 512)               16896 

In [20]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

18/18 [==============================] - 0s 882us/step


In [30]:
model.save("mae.h5")
X_test
model.predict(X_test[0].reshape(1,128))

1/1 [==============================] - 0s 13ms/step


array([[0.00097106, 0.00079864, 0.00115747, 0.00083965, 0.00090141,
        0.00121474, 0.00133817, 0.00120074, 0.0010057 , 0.00162539,
        0.00046757, 0.0008556 , 0.00170702, 0.0042268 , 0.0008161 ,
        0.00116268, 0.00099611, 0.00061055, 0.00074398, 0.00116903,
        0.00144293, 0.00120221, 0.000941  , 0.00183311, 0.00147641,
        0.00188156, 0.00201651, 0.00484275, 0.00116264, 0.00094404,
        0.00090496, 0.00076296, 0.0006266 , 0.00120573, 0.00105691,
        0.00118926, 0.0009052 , 0.0013784 , 0.00160251, 0.0011844 ,
        0.00233397, 0.00480825, 0.0007425 , 0.0005532 , 0.00077069,
        0.00054668, 0.00080642, 0.00069549, 0.00111785, 0.00087816,
        0.0009063 , 0.00152951, 0.001431  , 0.0015794 , 0.00343507,
        0.00584732, 0.00091676, 0.00063775, 0.00065861, 0.00093878,
        0.00196326, 0.00191831, 0.0021827 , 0.00225008, 0.00204468,
        0.00373286, 0.0037421 , 0.00355634, 0.00565538, 0.01039945,
        0.00107509, 0.00127017, 0.00105268, 0.00

In [21]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y

18/18 [==============================] - 0s 1ms/step - loss: 0.0643 - mae: 0.0643
[0.06433181464672089, 0.06433181464672089]
{'mse': 74.69402003850641, 'mean_absolute_error': 0.06433182287922903, 'mean_squared_log_error': 0.009416884413971045, 'median_absolute_error': 0.0031583952514693337, 'mean_absolute_percentage_error': 1086135949.754243}


array([[0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       ...,
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536],
       [0.00097106, 0.00079864, 0.00115747, ..., 0.0141268 , 0.01914554,
        0.03341536]], dtype=float32)